In [0]:
#importing the tools
from env import env
from src import utils, excel

import openpyxl
import pandas as pd

from pyspark.sql import functions as F
from datetime import datetime
from pyspark.sql.functions import date_format
from openpyxl.styles import NamedStyle

In [0]:
# Load the SA data
df_raw_sa = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(env["spec_advi_path"])
wb = openpyxl.load_workbook('Provider template.xlsx')
report_start = 'April 2022 to '

#display (spec_advi_path)
#df_raw_sa.display()

publishing_month = df_raw_sa.select(F.max("EROC_DerMonth")).collect()[0][0]
publishing_month = datetime.strptime(publishing_month, '%Y-%m-%d')
publishing_month = publishing_month.strftime("%B %Y")
date_header = (report_start + publishing_month) 

#print(date_header)

In [0]:
#Total requests for all types of SA
df_total_all_types_sa = df_raw_sa.where((F.col("EROC_DerMonth") > '2022-03-01') & (F.col("Acute_Status") == "Acute"))
    
df_total_all_types_sa = df_total_all_types_sa.groupby(
        "EROC_DerMonth",
        "RESP_Org_Code",
        "RESP_Org_Name"
    ).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
        "EROC_DerMonth",
        "RESP_Org_Code"
    )

# Select only the columns you need
df_total_all_types_sa = df_total_all_types_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# Show the result#display(df_total_all_types_sa)

In [0]:
#putting the data in pivot table format
df_total_all_types_sa_pivot = (df_total_all_types_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
        )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_total_all_types_sa_pivot)

In [0]:
#Processed requests for all types of SA
df_processed_all_types_sa = df_raw_sa.where((F.col("EROC_DerMonth") > '2022-03-01') & 
                                            (F.col("Acute_Status") == "Acute") & 
                                            (F.col("EROC_DerStatus") != 2) & 
                                            (F.col("EROC_DerOutcome") != 40))

df_processed_all_types_sa = df_processed_all_types_sa.groupby(
        "EROC_DerMonth",
        "RESP_Org_Code",
        "RESP_Org_Name"
    ).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
        "EROC_DerMonth",
        "RESP_Org_Code"
    )

# Select only the columns you need
df_processed_all_types_sa = df_processed_all_types_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

#display(df_processed_all_types_sa)

In [0]:
#putting the data in pivot table format
df_processed_all_types_sa_pivot = (df_processed_all_types_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
        )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_processed_all_types_sa_pivot)


In [0]:
#Diverted requests for all types of SA 
df_diverted_all_types_sa = df_raw_sa.where((F.col("EROC_DerMonth") > '2022-03-01') & 
                                            (F.col("Acute_Status") == "Acute") & 
                                            (F.col("EROC_DerStatus") != 2) & 
                                            (F.col("EROC_DerOutcome").isin (10,12)))

df_diverted_all_types_sa = df_diverted_all_types_sa.groupby(
        "EROC_DerMonth",
        "RESP_Org_Code",
        "RESP_Org_Name"
    ).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
        "EROC_DerMonth",
        "RESP_Org_Code"
    )

# Select only the columns you need
df_diverted_all_types_sa = df_diverted_all_types_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

#display(df_diverted_all_types_sa)

In [0]:
#putting the data in pivot table format
df_diverted_all_types_sa_pivot = (df_diverted_all_types_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
        )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_diverted_all_types_sa_pivot)

In [0]:
#Total requests for pre referral of SA
df_total_pre_referral_sa = df_raw_sa.where(((F.col("EROC_DerMonth") > '2022-03-01') & (F.col("Acute_Status") == "Acute")) & (F.col("EROC_DerTypeOfSpecialistAdvice") == 1))
    
df_total_pre_referral_sa = df_total_pre_referral_sa.groupby(
        "EROC_DerMonth",
        "RESP_Org_Code",
        "RESP_Org_Name"
    ).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
        "EROC_DerMonth",
        "RESP_Org_Code"
    )

# Select only the columns you need
df_total_pre_referral_sa = df_total_pre_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

#display(df_total_pre_referral_sa)

#putting the data in pivot table format
df_total_pre_referral_sa_pivot = (df_total_pre_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
        )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_total_pre_referral_sa_pivot)



In [0]:
# Processed requests for pre_referral SA
df_processed_pre_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    (F.col("EROC_DerStatus") != 2) & 
    (F.col("EROC_DerOutcome") != 40) & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 1)
)

df_processed_pre_referral_sa = df_processed_pre_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_processed_pre_referral_sa = df_processed_pre_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

#display(df_processed_pre_referral_sa)

# Putting the data in pivot table format
df_processed_pre_referral_sa_pivot = (df_processed_pre_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_processed_pre_referral_sa_pivot)

In [0]:
# Diverted requests for pre_referral SA 
df_diverted_pre_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    (F.col("EROC_DerStatus") != 2) & 
    (F.col("EROC_DerOutcome").isin([10, 12])) & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 1)
)

df_diverted_pre_referral_sa = df_diverted_pre_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_diverted_pre_referral_sa = df_diverted_pre_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# putting the data in pivot table format
df_diverted_pre_referral_sa_pivot = (df_diverted_pre_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name"
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name"
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_diverted_pre_referral_sa_pivot)

In [0]:
# Total requests for post referral of SA
df_total_post_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 2)
)

df_total_post_referral_sa = df_total_post_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_total_post_referral_sa = df_total_post_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# putting the data in pivot table format
df_total_post_referral_sa_pivot = (df_total_post_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_total_post_referral_sa_pivot)

In [0]:
# Processed requests for post_referral SA
df_processed_post_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    (F.col("EROC_DerStatus") != 2) & 
    (F.col("EROC_DerOutcome") != 40) & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 2)
)

df_processed_post_referral_sa = df_processed_post_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_processed_post_referral_sa = df_processed_post_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# putting the data in pivot table format
df_processed_post_referral_sa_pivot = (df_processed_post_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_processed_post_referral_sa_pivot)

In [0]:
# Diverted requests for post_referral SA 
df_diverted_post_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    (F.col("EROC_DerStatus") != 2) & 
    (F.col("EROC_DerOutcome").isin(10, 12)) & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 2)
)

df_diverted_post_referral_sa = df_diverted_post_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_diverted_post_referral_sa = df_diverted_post_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# putting the data in pivot table format
df_diverted_post_referral_sa_pivot = (df_diverted_post_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_diverted_post_referral_sa_pivot)

In [0]:
# Total requests for other_referral SA
df_total_other_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    #(F.col("EROC_DerStatus") != 2) & 
    #(F.col("EROC_DerOutcome") != 40) & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 99)
)

df_total_other_referral_sa = df_total_other_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_total_other_referral_sa = df_total_other_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# Get all unique RESP_Org_Code and RESP_Org_Name with Acute_Status == Acute
df_all_orgs = df_raw_sa.where(F.col("Acute_Status") == "Acute").select("RESP_Org_Code", "RESP_Org_Name").distinct()

# Join to ensure all RESP_Org_Code are included
df_total_other_referral_sa = df_all_orgs.join(
    df_total_other_referral_sa,
    on=["RESP_Org_Code", "RESP_Org_Name"],
    how="left"
).fillna(0, subset=["Requests"])

# putting the data in pivot table format
df_total_other_referral_sa_pivot = (df_total_other_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_total_other_referral_sa_pivot)

In [0]:
# Processed requests for other_referral SA

df_processed_other_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    (F.col("EROC_DerStatus") != 2) & 
    (F.col("EROC_DerOutcome") != 40) & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 99)
)

df_processed_other_referral_sa = df_processed_other_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_processed_other_referral_sa = df_processed_other_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# Get all unique RESP_Org_Code and RESP_Org_Name with Acute_Status == Acute
df_all_orgs = df_raw_sa.where(F.col("Acute_Status") == "Acute").select("RESP_Org_Code", "RESP_Org_Name").distinct()

# Join to ensure all RESP_Org_Code are included
df_processed_other_referral_sa = df_all_orgs.join(
    df_processed_other_referral_sa,
    on=["RESP_Org_Code", "RESP_Org_Name"],
    how="left"
).fillna(0, subset=["Requests"])

# putting the data in pivot table format
df_processed_other_referral_sa_pivot = (df_processed_other_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_processed_other_referral_sa_pivot)

In [0]:
# Diverted requests for other_referral SA
df_diverted_other_referral_sa = df_raw_sa.where(
    (F.col("EROC_DerMonth") > '2022-03-01') & 
    (F.col("Acute_Status") == "Acute") & 
    (F.col("EROC_DerStatus") != 2) & 
    (F.col("EROC_DerOutcome").isin(10, 12)) & 
    (F.col("EROC_DerTypeOfSpecialistAdvice") == 99)
)

df_diverted_other_referral_sa = df_diverted_other_referral_sa.groupby(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name"
).agg(F.sum("EROC_Requests").alias("Requests")).orderBy(
    "EROC_DerMonth",
    "RESP_Org_Code"
)

# Select only the columns you need
df_diverted_other_referral_sa = df_diverted_other_referral_sa.select(
    "EROC_DerMonth",
    "RESP_Org_Code",
    "RESP_Org_Name",
    "Requests"
)

# Get all unique RESP_Org_Code and RESP_Org_Name with Acute_Status == Acute
df_all_orgs = df_raw_sa.where(F.col("Acute_Status") == "Acute").select("RESP_Org_Code", "RESP_Org_Name").distinct()

# Join to ensure all RESP_Org_Code are included
df_diverted_other_referral_sa = df_all_orgs.join(
    df_diverted_other_referral_sa,
    on=["RESP_Org_Code", "RESP_Org_Name"],
    how="left"
).fillna(0, subset=["Requests"])

# putting the data in pivot table format
df_diverted_other_referral_sa_pivot = (df_diverted_other_referral_sa
    .groupby(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .pivot("EROC_DerMonth")
    .agg(F.sum("Requests"))
    .orderBy(
        "RESP_Org_Code",
        "RESP_Org_Name",
    )
    .withColumnRenamed("RESP_Org_Code", "Provider Code")
    .withColumnRenamed("RESP_Org_Name", "Provider Name")
)

#display(df_diverted_other_referral_sa_pivot)